In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import pandas as pd
import numpy as np
import json
import keras
import tensorflow as tf
import io
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.layers import Conv1D, Conv2D
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import LabelEncoder
import nltk
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from keras.layers import average
import tensorflow_hub as hub
from keras.layers import Average
from keras.layers import Concatenate
nltk.download('punkt')
from numpy import random
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
from keras.layers import SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import gc
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy
#plt.switch_backend('agg')
%matplotlib inline

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
!pip install keras_metrics
import keras_metrics as km

In [0]:
root_path='/content/drive/My Drive/offenseval/2020'

In [0]:
train_data=pd.read_csv(root_path+'/non_english_data/Turkish/offenseval-tr-training-v1.tsv',delimiter='\t')

In [0]:
print(train_data.head())

      id                                              tweet subtask_a
0  20948  @USER en güzel uyuyan insan ödülü jeon jungkoo...       NOT
1  10134  @USER Mekanı cennet olsun, saygılar sayın avuk...       NOT
2  23457  Kızlar aranızda kas yığını beylere düşenler ol...       NOT
3  18401  Biraz ders çalışayım. Tembellik ve uyku düşman...       NOT
4  17525  @USER Trezeguet yerine El Sharawy daha iyi olm...       NOT


In [0]:
print(train_data.shape)

(31277, 3)


## Tokenization and Split

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from sklearn.model_selection import StratifiedKFold
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)

In [0]:
df_train,df_dev=train_test_split(train_data,test_size=0.1)

In [0]:
y_train=(df_train['subtask_a']=="OFF").astype(int)
y_dev=(df_dev['subtask_a']=="OFF").astype(int)

In [0]:
print(train_data.shape)

(31277, 3)


In [0]:
max_words =10000
max_len = 50
tok = Tokenizer(max_words)
tok.fit_on_texts(train_data['tweet'].astype(str))

In [0]:
# sequences_train = tok.texts_to_sequences(df_train['tweet'].astype(str))
# vocab_size = len(tok.word_index) + 1
# sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len,padding='post',truncating='post')

In [0]:
# sequences_dev = tok.texts_to_sequences(df_dev['tweet'].astype(str))
# vocab_size = len(tok.word_index) + 1
# sequences_matrix_dev = sequence.pad_sequences(sequences_dev,maxlen=max_len,padding='post',truncating='post')

In [0]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'custom_gelu': Activation(custom_gelu)})

## Model

In [0]:
word_index=tok.word_index

In [0]:
print(word_index)

{'user': 1, 'bir': 2, 'bu': 3, 've': 4, 'çok': 5, 'de': 6, 'ne': 7, 'da': 8, 'için': 9, 'ama': 10, 'o': 11, 'ben': 12, 'var': 13, 'kadar': 14, 'gibi': 15, 'ya': 16, 'yok': 17, 'bi': 18, 'daha': 19, 'diye': 20, 'her': 21, 'değil': 22, 'ki': 23, 'en': 24, 'sen': 25, 'mi': 26, 'güzel': 27, 'olsun': 28, 'sonra': 29, 'iyi': 30, 'şey': 31, 'nasıl': 32, 'bile': 33, 'artık': 34, 'hiç': 35, 'böyle': 36, 'mı': 37, 'ile': 38, 'benim': 39, 'bana': 40, 'olan': 41, 'zaman': 42, 'olur': 43, 'olarak': 44, 'şu': 45, 'yine': 46, 'seni': 47, 'beni': 48, 'gün': 49, 'sadece': 50, 'sana': 51, 'biz': 52, '1': 53, 'öyle': 54, 'allah': 55, 'zaten': 56, '2': 57, 'senin': 58, 'şimdi': 59, 'tek': 60, 'oldu': 61, 'bunu': 62, 'neden': 63, 'siz': 64, 'büyük': 65, 'önce': 66, 'insan': 67, '3': 68, 'hep': 69, 'kendi': 70, 'aynı': 71, 'çünkü': 72, 'başka': 73, 'adam': 74, 'bugün': 75, 'bizim': 76, 'herkes': 77, 'son': 78, 'sizin': 79, 'biri': 80, 'biraz': 81, 'devam': 82, 'not': 83, 'mu': 84, 'bence': 85, 'ilk': 86, '5

In [0]:
!pip install keras-self-attention

In [0]:
from keras.layers import Concatenate
from sklearn.metrics import precision_recall_fscore_support

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras_self_attention import SeqSelfAttention
from keras.layers import CuDNNGRU,CuDNNLSTM,GlobalMaxPooling1D,GlobalAveragePooling1D
from sklearn.utils import class_weight
class Attention(Layer):
    def __init__(self,step_dim=20,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, Dropout

#max_len=

def BidLstm(maxlen, max_features, embed_size):
    inp1 = Input(shape=(maxlen, ))
    #inp2=Input(shape=(1,))
    x1=Embedding(max_words+1,embed_size)(inp1)
    #x1 = Embedding(max_words + 1,embed_size,weights=[embedding_matrix_1],
    #              trainable=True)(inp1)
    # x2 = Embedding(len(tok.word_index) + 1,embed_size_2,weights=[embedding_matrix_2],
    #                trainable=True)(inp1)
    # x3 = Embedding(len(tok.word_index) + 1,embed_size_3,weights=[embedding_matrix_3],
    #                trainable=True)(inp1)
    # x1 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x1)
    # x2 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x2)
    # x3 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x3)   
    #x1 = Attention(maxlen)(x1)
    # x2 = Attention(maxlen)(x2)
    # x3 = Attention(maxlen)(x3)
    # x=  Concatenate()([x1,x2,x3])
    x1 = CuDNNLSTM(200, return_sequences=True)(x1)   
    x2=  GlobalMaxPooling1D()(x1)
    x3= GlobalAveragePooling1D()(x1)
    x=  Concatenate()([x2,x3])
    x = Dropout(0.1)(x)
    #x = Attention(maxlen)(x)
    # layer = Dense(600,name='FC1')(x)
    # layer = Dense(300,activation='relu')(layer)
    layer = Dense(128,activation='relu')(x)
 #   layer = BatchNormalization(name = 'BN1')(layer)
    #layer = Activation('relu')(layer)
    #layer = Dropout(0.4)(layer)
    layer = Dense(64,name='FC2')(layer)
#    layer = BatchNormalization(name = 'BN2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.4)(layer)
   # layer=  Concatenate()([layer,inp2])
    # layer=Dense(256,activation='relu')(layer)
    # layer=Dense(128,activation='relu')(layer)
    layer = Dense(1,name='out_layer',activation='sigmoid')(layer)

    model = Model(inputs=[inp1],outputs=layer)

    return model
model_bi=BidLstm(max_len,max_features=max_words,embed_size=100)

In [0]:
model_bi.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc',km.f1_score()])

In [0]:
model_bi.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 50, 100)      1000100     input_4[0][0]                    
__________________________________________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)        (None, 50, 200)      241600      embedding_4[0][0]                
__________________________________________________________________________________________________
global_max_pooling1d_4 (GlobalM (None, 200)          0           cu_dnnlstm_4[0][0]               
____________________________________________________________________________________________

In [0]:
y=(train_data['subtask_a']=="OFF").astype(int)

In [0]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(y),y)
class_weights=dict(enumerate(class_weights))

In [0]:
# print(class_weights)

In [0]:
cp_filepath='/content/drive/My Drive/offenseval/'+'checkpoints/lstm_model_2020a_turkish.h5'
cp_check_point=keras.callbacks.ModelCheckpoint(cp_filepath, monitor='val_f1_score', verbose=0, save_best_only=True, save_weights_only=False, mode='max', period=1)
es = EarlyStopping(monitor='val_f1_score', mode='max', min_delta=0,patience=2,restore_best_weights=True)
reduce_lr=keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [0]:
# model_bi.fit([sequences_matrix_train],y_train,validation_data=([sequences_matrix_dev],y_dev),epochs=10,batch_size=32,class_weight=class_weights,callbacks=[es,cp_check_point])

In [0]:
# y_preds_dl=model_bi.predict(sequences_matrix_dev)

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
#y_pred = model_bi.predict(X_dev, batch_size=30, verbose=1)

# print(classification_report(y_dev, y_preds_dl.round()))

In [0]:
# train_data.head()

In [0]:

# sequences_train = tok.texts_to_sequences(df_train['tweet'].astype(str))
# vocab_size = len(tok.word_index) + 1
# sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len,padding='post',truncating='post')

In [0]:
f1s=[]
for train_ind,dev_ind in kfold.split(train_data['tweet'],y):
  #print(train_data['tweet'][train_ind].shape)
  x_train=train_data['tweet'][train_ind]
  y_train=y[train_ind]
  x_dev=train_data['tweet'][dev_ind]
  y_dev=y[dev_ind]
  seq_train=tok.texts_to_sequences(x_train.astype(str))
  seq_matrix_train=sequence.pad_sequences(seq_train,maxlen=max_len,padding='post',truncating='post')
  seq_dev=tok.texts_to_sequences(x_dev.astype(str))
  seq_matrix_dev=sequence.pad_sequences(seq_dev,maxlen=max_len,padding='post',truncating='post')
  model_bi.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc',km.f1_score()])
  model_bi.fit([seq_matrix_train],y_train,validation_split=0.1,epochs=10,batch_size=32,class_weight=class_weights,callbacks=[es,cp_check_point])
  scores = model_bi.evaluate(seq_matrix_dev, y_dev, verbose=0)
  pred = model_bi.predict(seq_matrix_dev)
  print(classification_report(y_dev, pred.round()))
  f1s.append(f1_score(y_dev,pred.round(),average='macro'))
  #f1s.append(scores[2])
  print(scores)
print("Mean f1:{0}, standard dev:{1}".format(np.mean(f1s),np.std(f1s)))

Train on 25334 samples, validate on 2815 samples
Epoch 1/10
25334/25334 [==============================] - 10s 413us/step - loss: 0.5866 - acc: 0.7073 - f1_score: 0.4553 - val_loss: 0.5137 - val_acc: 0.7925 - val_f1_score: 0.5487
Epoch 2/10
25334/25334 [==============================] - 9s 345us/step - loss: 0.4279 - acc: 0.8244 - f1_score: 0.6300 - val_loss: 0.5479 - val_acc: 0.8053 - val_f1_score: 0.5609
Epoch 3/10
25334/25334 [==============================] - 9s 346us/step - loss: 0.3193 - acc: 0.8664 - f1_score: 0.7155 - val_loss: 0.6920 - val_acc: 0.7943 - val_f1_score: 0.5379
Epoch 4/10
25334/25334 [==============================] - 9s 359us/step - loss: 0.2165 - acc: 0.9120 - f1_score: 0.8018 - val_loss: 0.7182 - val_acc: 0.7780 - val_f1_score: 0.5297
              precision    recall  f1-score   support

           0       0.90      0.85      0.87      2524
           1       0.48      0.60      0.53       604

    accuracy                           0.80      3128
   macro avg

In [0]:
from sklearn.metrics import f1_score

In [0]:
pred = model_bi.predict(seq_matrix_dev)
print(f1_score(y_dev,pred.round(),average='macro'))
print(classification_report(y_dev, pred.round()))

0.9499153395570199
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2523
           1       0.91      0.92      0.92       604

    accuracy                           0.97      3127
   macro avg       0.95      0.95      0.95      3127
weighted avg       0.97      0.97      0.97      3127



## ML algos

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?' ]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

print(X.shape)

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
(4, 9)


In [0]:
vectorizer =TfidfVectorizer(max_features=10000,lowercase=True, analyzer='word',
                        stop_words= 'english',ngram_range=(1,5),dtype=np.float32,max_df=0.5,min_df=2)
vectorizer.fit(train_data['tweet'].astype(str))
# x_train=vectorizer.transform(['clean_col'].astype(str))
# x_dev=vectorizer.transform(dev_data['clean_col'].astype(str))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float32'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.5, max_features=10000,
                min_df=2, ngram_range=(1, 5), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
from sklearn.naive_bayes import MultinomialNB
# clf = MultinomialNB().fit(x_train, labels_train_le)
# y_preds=clf.predict(x_dev)
# f1_score(labels_dev_le,y_preds,average='macro')

In [0]:
from sklearn.linear_model import SGDClassifier
# sgd_clf=SGDClassifier().fit(x_train,labels_train_le)
# y_preds=sgd_clf.predict(x_dev)
# f1_score(labels_dev_le,y_preds,average='macro')

In [0]:
from sklearn.svm import SVC
# clf_svc=SVC(gamma='scale',decision_function_shape='ovo').fit(x_train,labels_train_le)
# y_preds=clf_svc.predict(x_dev)
# f1_score(labels_dev_le,y_preds,average='macro')

In [0]:
f1s_nb=[]
f1s_sgd=[]
f1s_svc=[]
for train_ind,dev_ind in kfold.split(train_data['tweet'],y):
  #print(train_data['tweet'][train_ind].shape)
  x_train=train_data['tweet'][train_ind]
  y_train=y[train_ind]
  x_dev=train_data['tweet'][dev_ind]
  y_dev=y[dev_ind]
  x_train=vectorizer.transform(x_train.astype(str))
  x_dev=vectorizer.transform(x_dev.astype(str))
  clf = MultinomialNB().fit(x_train, y_train)
  y_preds=clf.predict(x_dev)
  #print(f1_score(y_dev,y_preds,average='macro'))
  f1s_nb.append(f1_score(y_dev,y_preds,average='macro'))
  #f1s.append(scores[2])
  print(scores)
print("Mean f1:{0}, standard dev:{1}".format(np.mean(f1s_nb),np.std(f1s_nb)))

[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
Mean f1:0.5834355395986814, standard dev:0.007608792743137974


In [0]:
for train_ind,dev_ind in kfold.split(train_data['tweet'],y):
  #print(train_data['tweet'][train_ind].shape)
  x_train=train_data['tweet'][train_ind]
  y_train=y[train_ind]
  x_dev=train_data['tweet'][dev_ind]
  y_dev=y[dev_ind]
  x_train=vectorizer.transform(x_train.astype(str))
  x_dev=vectorizer.transform(x_dev.astype(str))
  sgd_clf=SGDClassifier().fit(x_train,y_train)
  y_preds=sgd_clf.predict(x_dev)
  #print(f1_score(y_dev,y_preds,average='macro'))
  f1s_sgd.append(f1_score(y_dev,y_preds,average='macro'))
  #f1s.append(scores[2])
  print(scores)
print("Mean f1:{0}, standard dev:{1}".format(np.mean(f1s_sgd),np.std(f1s_sgd)))

[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
Mean f1:0.5913309385766453, standard dev:0.009822057525018916


In [0]:
for train_ind,dev_ind in kfold.split(train_data['tweet'],y):
  #print(train_data['tweet'][train_ind].shape)
  x_train=train_data['tweet'][train_ind]
  y_train=y[train_ind]
  x_dev=train_data['tweet'][dev_ind]
  y_dev=y[dev_ind]
  x_train=vectorizer.transform(x_train.astype(str))
  x_dev=vectorizer.transform(x_dev.astype(str))
  clf_svc=SVC(gamma='scale',decision_function_shape='ovo').fit(x_train,y_train)
  y_preds=clf_svc.predict(x_dev)
  #print(f1_score(y_dev,y_preds,average='macro'))
  f1s_svc.append(f1_score(y_dev,y_preds,average='macro'))
  #f1s.append(scores[2])
  print(f1_score(y_dev,y_preds,average='macro'))
print("Mean f1:{0}, standard dev:{1}".format(np.mean(f1s_svc),np.std(f1s_svc)))

[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
[0.23058211398453732, 0.9686600577347112, 0.9192750734082628]
Mean f1:0.6153654050253323, standard dev:0.009657411433516862
